https://microsoft.github.io/autogen/docs/topics/non-openai-models/local-lm-studio/

one way conversation
two conversations

In [2]:
gemma = {
    "config_list": [
        {
            "model": "lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

phi2 = {
    "config_list": [
        {
            "model": "TheBloke/phi-2-GGUF/phi-2.Q4_K_S.gguf:0",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

mistral = {
    "config_list": [
        {
            "model": "TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
            "base_url": "http://localhost:1234/v1",
            "api_key": "lm-studio",
        },
    ],
    "cache_seed": None,  # Disable caching.
}

In [3]:
from autogen import ConversableAgent

teamhead = ConversableAgent(
    "teamhead",
    llm_config=mistral,
    system_message="you are a professional team leader of \n"
    "a team of stock analyst, you instruct general analytical\n"
    "frame work and ask analysts to process data and give investment\n"
    "strategy and suggestions\n"
    "in the end you combine opinions from both data analyst and news analyst\n",
)
analystondata = ConversableAgent(
    "analystondata",
    llm_config=mistral,
    system_message="you are a professional data analyst \n"
    "who can retrive financial data according to request \n"
    "and analyze trend, plot line graph, give price prediction \n"
    "make buy or sell recommendation based on historical data\n",
)
analystonnews = ConversableAgent(
    "analystonnews",
    llm_config=mistral,
    system_message="you are a professional news analyst \n"
    "who can retrive financial news data according to request \n"
    "and analyze trend, risk, opportunities \n"
    "make buy or sell recommendation based on the summarized info\n",
)

In [4]:
chat_result = teamhead.initiate_chat(analystondata, message="analyze AAPL", max_turns=2)
chat_result = teamhead.initiate_chat(analystonnews, message="analyze AAPL", max_turns=2)

teamhead (to analystondata):

analyze AAPL

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
analystondata (to teamhead):

 I'm an AI language model and don't have the ability to retrieve real-time financial data or make buy/sell recommendations directly. However, I can provide you with a general analysis of Apple Inc. (AAPL) based on historical data that is widely available.

Apple Inc. is a leading technology company headquartered in Cupertino, California. Its stock symbol is "AAPL" and it is listed on the NASDAQ exchange. Here's an analysis of AAPL using some key financial indicators:

1. Historical Stock Price: The historical stock price for AAPL shows a clear uptrend over the past decade, with significant volatility. However, the overall trend has been positive, with occasional dips that have been bought up by investors.
2. Earnings Per Share (EPS): Apple's EPS has been steadily increasing over the past several years. T

https://stackoverflow.com/questions/77376267/how-can-i-get-multiple-autogen-agents-to-work-together

In [14]:
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import UserProxyAgent

In [21]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="TERMINATE",
    max_consecutive_auto_reply=5,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={"work_dir": "web", "use_docker": False},  # Adding "use_docker": False
    llm_config=mistral,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
    Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)


RuntimeError: Code execution is set to be run in docker (default behaviour) but docker is not running.
The options available are:
- Make sure docker is running (advised approach for code execution)
- Set "use_docker": False in code_execution_config
- Set AUTOGEN_USE_DOCKER to "0/False/no" in your environment variables
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

In [27]:
groupchat = GroupChat(agents=[teamhead,analystondata, analystonnews], messages=[], max_round=6)
manager = GroupChatManager(groupchat=groupchat, llm_config=mistral)

# Prompting the user for input
prompt = "Enter your message: "
user_input = input(prompt)

# Constructing the message object with format and user input
message = {
    "content": f"analyze {user_input}"
}

# Passing the message object to initiate_chat()
user_proxy.initiate_chat(manager, message)



user_proxy (to chat_manager):

analyze amazon stock

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
analystondata (to chat_manager):

 To analyze Amazon's (AMZN) stock, I would first retrieve the historical financial data for the company from reliable sources such as Yahoo Finance or Google Finance. I would then use a data analysis tool or programming language like Python to perform the analysis.

Here are some steps I would take to analyze Amazon's stock:

1. Data Retrieval: I would retrieve the historical stock price data for Amazon, including the opening price, closing price, high, low, and volume for each trading day over a specified time period (e.g., 5 years).

2. Data Visualization: I would plot a line graph of the closing prices to visualize the trend in Amazon's stock price over the given time frame.

3. Trend Analysis: I would analyze the trend of Amazon's stock price by calculating key performance indicators (KP

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:


Based on the given role play scenario, the next role from ['teamhead', 'analystondata', 'analystonnews'] to play is 'analystondata'.



>>>>>>>> USING AUTO REPLY...
analystonnews (to chat_manager):

 Yes, you're absolutely right. It's important to remember that historical data provides context and trends but does not guarantee future performance. As a professional data analyst, it's essential to provide accurate, unbiased, and transparent analysis while complying with ethical standards and regulations. Clear communication of findings and recommendations is crucial for informed decision-making by clients or stakeholders.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
analystondata (to chat_manager):

 Absolutely. Providing clear, unbiased, and transparent analysis is crucial in maintaining trust with clients or stakeholders and ensuring that they can make informed decisions based on accurate information. Ethical standards and regulations are also essential to ensure that the analysis is conducted fairly and without any conflicts of interest or misrepresen

ChatResult(chat_id=None, chat_history=[{'content': 'analyze amazon stock', 'role': 'assistant'}], summary='analyze amazon stock', cost={'usage_including_cached_inference': {'total_cost': 0, 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q3_K_S.gguf': {'cost': 0, 'prompt_tokens': 256, 'completion_tokens': 256, 'total_tokens': 512}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q3_K_S.gguf': {'cost': 0, 'prompt_tokens': 256, 'completion_tokens': 256, 'total_tokens': 512}}}, human_input=['analyze amazon stock'])

BadRequestError: Error code: 400 - {'error': "'messages' array must only contain objects with a 'content' field that is not empty."}

In [ ]:
groupchat = GroupChat(agents=[teamhead, analystondata, analystonnews], messages=[], max_round=6)
manager = GroupChatManager(groupchat=groupchat, llm_config=mistral)

# Prompting the user for input
prompt = "Enter your message: "
user_input = input(prompt)

# Constructing the message object with format and user input
message = {
    "content": f"analyze {user_input}"
}

# Passing the message object to initiate_chat()
user_proxy.initiate_chat(manager, message)
